In [ ]:
We need the ball to be at about 5 further than the ramp.
Have all types of ramps in width and height.
Have positive reward for going up.

In [ ]:
# Objects to place. Need ramp, platform, agent, good goals all around top
# Platform dimensions will be identical to ramp
base = """
!ArenaConfig
arenas:
  0: !Arena
    pass_mark: 2
    t: 250
    items:
    - !Item
      name: Agent
      positions:
      - !Vector3 {x: 1, y: 0, z: 1}
      rotations: [45]
    - !Item
      name: Wall
      positions:
      - !Vector3 {x: 20, y: 0, z: 20}
      rotations: [0]
      colors: 
      - !RGB {r: 153, g: 153, b: 153}
      sizes:
      - !Vector3 {x: 5, y: 3, z: 5}
    - !Item
      name: GoodGoal
      positions:
      - !Vector3 {x: 20, y: 4, z: 20}      
      sizes:
      - !Vector3 {x: 3, y: 3, z: 3}
    - !Item
      name: Ramp
      positions: 
      - !Vector3 {x: 20, y: 0, z: 15}
      rotations: [180]
      sizes:
      - !Vector3 {x: 5, y: 3, z: 5}
      colors:
      - !RGB {r: 255, g: 0, b: 255}"""

In [ ]:
from random import randrange, choice
import numpy as np
locs = {
    'agent_x':0,
    'agent_z':0,
    'agent_rot':0,
    'goals':"",
    'ramp_x':0,
    'ramp_z':0,
    'ramp_height':0,
    'ramp_width':0,
    'ramp_depth':0,
    
}
def valid_loc(diff=0, num_goals=2,side='left'):
    res = locs.copy()
    agent_x = randrange(5,35)
    agent_z = randrange(1,10-diff)
    wall_tall = randrange(2,5)
    wall_x = agent_x + choice([1,-1])*randrange(diff*4,1+diff*8)
    wall_x = max(min(wall_x,35-diff*2),5+diff*2)
    wall_z = agent_z + randrange(6+diff,7+diff*4)
    if wall_z>39:
        wall_z = 35
    wall_size = randrange(1+diff*3,3+diff*5)
    if (wall_x + wall_size/2) > 35:
        wall_size = 39-wall_x
    if (wall_x-wall_size/2)<5:
        wall_size = int(wall_x/2-1)
        if wall_size<2:
            wall_size=2
    wall_deep = 2+ diff
    goals = gen_goals(wall_x, wall_z,wall_size, wall_deep,diff, num_goals, side)
    wall_diff = wall_x - agent_x
    if wall_diff>0: # wall is to the right
        agent_rot = int(60*wall_diff/40)
    else: # wall is to the left
        agent_rot = int(360+(60*wall_diff/40))
        
    for k in locs:
        locs[k] = locals()[k]
    return locs
a_goal = """
        - !Item
          name: GoodGoal
          positions:
          - !Vector3 {{x: {x}, y: 0, z: {z}}}
          sizes:
          - !Vector3 {{x: 1, y: 1, z: 1}}"""
def gen_goals(wall_x, wall_z,wall_size, wall_deep,diff, num=2, side='left'):
    """Num is num of goals on each side of wall"""
    res = []
    if side=='left':
        sign = -1
    else:
        sign = 1
    for i in range(0,num):
        specs = {
            'x': wall_x+(sign*((wall_size/2)+0.5+i))-sign*max(0,min(diff-1,3)),
            'z': wall_z+wall_deep/2+1 if diff >1 else wall_z
        }
        res.append(a_goal.format(**specs))
    return "".join(res)

num_arenas = 60
curric_num = 14
for i in range(0,num_arenas):
    
    locs = valid_loc(int(i/10), side='left')
    if any((v>39)|(v<0) for k,v in locs.items() if k not in ['agent_rot','goals']):
        raise Exception(f"BAD ENTRY {locs}")
    arena = """!ArenaConfig
    arenas:
      -1: !Arena
        items:
        - !Item
          name: Agent
          positions:
          - !Vector3 {{x: {agent_x}, y: 0, z: {agent_z}}}
          rotations: [{agent_rot}]{goals}
        - !Item
          name: Wall
          positions:
          - !Vector3 {{x: {wall_x}, y: 0, z: {wall_z}}}
          sizes:
          - !Vector3 {{x: {wall_size}, y: {wall_tall}, z: {wall_deep}}}
          rotations: [0]
          colors: 
          - !RGB {{r: 153, g: 153, b: 153}}
        pass_mark: 0
        t: 250""".format(**locs)
    text_file = open(f"../configurations/curriculum{curric_num}/{i}.yml", "w")
    text_file.write(arena)
    text_file.close()
    text_file = open(f"../configurations/curriculum{curric_num}/{i+60}.yml", "w")
    text_file.write(arena)
    text_file.close()